In [36]:
import sys

In [37]:
import dateparser
import toolz as tz
from lenses import lens
import re

In [38]:
r = '''归集日期 期初剩余本金 回收本金 回收利息 期末剩余本金
2024/2/29 4,500,001,047.85 224,053,881.53 4,793,572.11 4,275,947,166.32
2024/3/31 4,275,947,166.32 224,371,124.30 8,756,707.23 4,051,576,042.02
2024/4/30 4,051,576,042.02 224,593,099.27 8,437,800.01 3,826,982,942.75
2024/5/31 3,826,982,942.75 224,675,644.80 8,117,628.44 3,602,307,297.95
2024/6/30 3,602,307,297.95 224,782,519.51 7,796,819.43 3,377,524,778.44
2024/7/31 3,377,524,778.44 224,853,790.05 7,474,909.07 3,152,670,988.39
2024/8/31 3,152,670,988.39 225,037,373.80 7,152,615.30 2,927,633,614.59
2024/9/30 2,927,633,614.59 204,861,136.19 6,829,261.23 2,722,772,478.40
2024/10/31 2,722,772,478.40 183,623,456.87 6,544,089.27 2,539,149,021.53
2024/11/30 2,539,149,021.53 165,158,145.90 6,297,520.59 2,373,990,875.63
2024/12/31 2,373,990,875.63 165,280,964.01 6,085,006.84 2,208,709,911.62
2025/1/31 2,208,709,911.62 165,344,301.09 5,871,663.54 2,043,365,610.53
2025/2/28 2,043,365,610.53 165,517,701.66 5,657,672.26 1,877,847,908.87
2025/3/31 1,877,847,908.87 165,689,760.88 5,442,696.27 1,712,158,147.99
2025/4/30 1,712,158,147.99 165,824,161.90 5,226,688.42 1,546,333,986.09
2025/5/31 1,546,333,986.09 165,929,663.79 5,009,891.19 1,380,404,322.30
2025/6/30 1,380,404,322.30 165,736,745.90 4,792,373.25 1,214,667,576.40
2025/7/31 1,214,667,576.40 159,276,466.51 4,575,254.87 1,055,391,109.89
2025/8/31 1,055,391,109.89 147,461,959.62 4,359,158.27 907,929,150.27
2025/9/30 907,929,150.27 99,263,141.11 4,144,590.07 808,666,009.16
2025/10/31 808,666,009.16 48,645,772.46 3,941,471.02 760,020,236.70
2025/11/30 760,020,236.70 47,896,429.34 3,751,825.80 712,123,807.36
2025/12/31 712,123,807.36 47,726,828.72 3,566,454.98 664,396,978.64
2026/1/31 664,396,978.64 47,465,075.55 3,381,783.93 616,931,903.09
2026/2/28 616,931,903.09 47,409,880.58 3,198,333.02 569,522,022.51
2026/3/31 569,522,022.51 47,151,593.98 3,015,125.23 522,370,428.53
2026/4/30 522,370,428.53 46,752,666.34 2,831,447.56 475,617,762.19
2026/5/31 475,617,762.19 42,131,461.47 2,647,671.02 433,486,300.72
2026/6/30 433,486,300.72 36,321,805.32 2,463,932.26 397,164,495.40
2026/7/31 397,164,495.40 34,478,801.14 2,283,847.01 362,685,694.26
2026/8/31 362,685,694.26 33,254,224.50 2,107,820.76 329,431,469.76
2026/9/30 329,431,469.76 25,919,909.57 1,936,319.15 303,511,560.19
2026/10/31 303,511,560.19 18,266,387.54 1,795,012.87 285,245,172.65
49
归集日期 期初剩余本金 回收本金 回收利息 期末剩余本金
2026/11/30 285,245,172.65 18,344,444.41 1,686,863.94 266,900,728.24
2026/12/31 266,900,728.24 18,124,788.71 1,578,233.20 248,775,939.53
2027/1/31 248,775,939.53 17,652,452.42 1,470,958.15 231,123,487.11
2027/2/28 231,123,487.11 17,101,008.47 1,366,604.67 214,022,478.64
2027/3/31 214,022,478.64 16,750,648.94 1,265,667.54 197,271,829.70
2027/4/30 197,271,829.70 16,519,545.91 1,166,898.05 180,752,283.79
2027/5/31 180,752,283.79 16,290,573.62 1,069,552.90 164,461,710.17
2027/6/30 164,461,710.17 15,697,217.00 973,620.80 148,764,493.17
2027/7/31 148,764,493.17 15,175,620.76 881,350.59 133,588,872.41
2027/8/31 133,588,872.41 14,893,840.24 792,302.20 118,695,032.17
2027/9/30 118,695,032.17 13,846,448.68 704,969.06 104,848,583.49
2027/10/31 104,848,583.49 12,871,362.28 624,381.81 91,977,221.21
2027/11/30 91,977,221.21 12,184,924.68 550,076.62 79,792,296.53
2027/12/31 79,792,296.53 11,372,715.34 480,057.86 68,419,581.19
2028/1/31 68,419,581.19 10,417,602.08 415,091.02 58,001,979.11
2028/2/29 58,001,979.11 10,058,471.29 356,057.82 47,943,507.82
2028/3/31 47,943,507.82 9,516,792.48 299,185.83 38,426,715.34
2028/4/30 38,426,715.34 8,691,893.24 244,682.20 29,734,822.10
2028/5/31 29,734,822.10 7,508,051.18 193,802.39 22,226,770.92
2028/6/30 22,226,770.92 6,861,375.88 147,179.69 15,365,395.04
2028/7/31 15,365,395.04 6,246,932.67 102,972.05 9,118,462.37
2028/8/31 9,118,462.37 5,672,073.39 61,774.98 3,446,388.98
2028/9/30 3,446,388.98 3,446,388.98 23,561.96 -'''

In [39]:
def rmDigitsInLine(x):
    return re.sub("\n\d+\n","",x)

def rmHeader(x):
    h1 = "归集日期 期初剩余本金 回收本金 回收利息 期末剩余本金"
    return re.sub(h1,"",x)

def removeComma(x):
    return x.replace(",","")

def hyphenToZero(x):
    return x.replace("-","0")

def removeEmpty(x):
    return [ _ for _ in x if _]

def splitSpace(xs:list):
    return [ _.split(" ") for _ in xs]

def normalizedDate(x:str):
    return dateparser.parse(x).strftime("%Y-%m-%d")

In [40]:
tz.pipe(r
         ,rmDigitsInLine
         ,rmHeader
         ,hyphenToZero
         ,removeComma
         ,lambda x: x.split("\n")
         ,removeEmpty
         ,splitSpace
         ,lambda x: x & lens.Each()[0].modify(normalizedDate)
         ,lambda x: x & lens.Each().modify(lambda xs: [xs[0]]+[xs[4]]+xs[2:4])
         )

[['2024-02-29', '4275947166.32', '224053881.53', '4793572.11'],
 ['2024-03-31', '4051576042.02', '224371124.30', '8756707.23'],
 ['2024-04-30', '3826982942.75', '224593099.27', '8437800.01'],
 ['2024-05-31', '3602307297.95', '224675644.80', '8117628.44'],
 ['2024-06-30', '3377524778.44', '224782519.51', '7796819.43'],
 ['2024-07-31', '3152670988.39', '224853790.05', '7474909.07'],
 ['2024-08-31', '2927633614.59', '225037373.80', '7152615.30'],
 ['2024-09-30', '2722772478.40', '204861136.19', '6829261.23'],
 ['2024-10-31', '2539149021.53', '183623456.87', '6544089.27'],
 ['2024-11-30', '2373990875.63', '165158145.90', '6297520.59'],
 ['2024-12-31', '2208709911.62', '165280964.01', '6085006.84'],
 ['2025-01-31', '2043365610.53', '165344301.09', '5871663.54'],
 ['2025-02-28', '1877847908.87', '165517701.66', '5657672.26'],
 ['2025-03-31', '1712158147.99', '165689760.88', '5442696.27'],
 ['2025-04-30', '1546333986.09', '165824161.90', '5226688.42'],
 ['2025-05-31', '1380404322.30', '165929

In [41]:
r 

'归集日期 期初剩余本金 回收本金 回收利息 期末剩余本金\n2024/2/29 4,500,001,047.85 224,053,881.53 4,793,572.11 4,275,947,166.32\n2024/3/31 4,275,947,166.32 224,371,124.30 8,756,707.23 4,051,576,042.02\n2024/4/30 4,051,576,042.02 224,593,099.27 8,437,800.01 3,826,982,942.75\n2024/5/31 3,826,982,942.75 224,675,644.80 8,117,628.44 3,602,307,297.95\n2024/6/30 3,602,307,297.95 224,782,519.51 7,796,819.43 3,377,524,778.44\n2024/7/31 3,377,524,778.44 224,853,790.05 7,474,909.07 3,152,670,988.39\n2024/8/31 3,152,670,988.39 225,037,373.80 7,152,615.30 2,927,633,614.59\n2024/9/30 2,927,633,614.59 204,861,136.19 6,829,261.23 2,722,772,478.40\n2024/10/31 2,722,772,478.40 183,623,456.87 6,544,089.27 2,539,149,021.53\n2024/11/30 2,539,149,021.53 165,158,145.90 6,297,520.59 2,373,990,875.63\n2024/12/31 2,373,990,875.63 165,280,964.01 6,085,006.84 2,208,709,911.62\n2025/1/31 2,208,709,911.62 165,344,301.09 5,871,663.54 2,043,365,610.53\n2025/2/28 2,043,365,610.53 165,517,701.66 5,657,672.26 1,877,847,908.87\n2025/3/31 1,877,8